# Remote Renewable Energy Hub

## The aim of this notebook is to launch Jocelyn's Belgium model 


In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

belgian_data_folder = os.path.join(project_root, 'Data', 'Belgian_model')
plot_folder = os.path.join(project_root, 'Plots')
simulation_folder = os.path.join(project_root, 'Simulations')
result_folder = os.path.join(project_root, 'Results')
template_folder = os.path.join(project_root, 'Templates')
model_folder = os.path.join(project_root, 'Models GBOML')

 
sys.path.append(os.path.join(project_root, 'Modules'))

import GBOML_function as gf
import process_funct as pf

current_dir = os.getcwd()
result_dir = os.path.join(current_dir, 'Results')

# Set up

### Class and functions

In [2]:
import argparse
import sys
import json
import time
from termcolor import colored
import logging
import sys
import pandas as pd
from tqdm import tqdm
from copy import deepcopy 
from gboml import GbomlGraph
from gboml.output import write_csv
from gboml.compiler import parse_file
from gboml.compiler.classes import Parameter, Expression, Constraint, Identifier, Node

class MyGbomlGraph(GbomlGraph):
    def update_global_parameter(self, param: Parameter):
        self.global_parameters = [x if x.name != param.name else param for x in self.global_parameters]
    def get_node_by_name(self, name: str):
        for node in self.list_nodes:
            if node.name == name:
                return node
        return None
    def get_hyperedge_by_name(self, name: str):
        for h in self.list_hyperedges:
            if h.name == name:
                return h
        return None
    
    def export_model_to_lp(self, filename: str):
        """
        Optional method: exports model to LP format (requires Gurobi model to be built first).
        """
        if hasattr(self, 'model'):  # Gurobi model exists after build_model()
            self.model.write(filename)
            print(f"✅ Model written to {filename}")
        else:
            print("⚠️ Gurobi model not found. Make sure build_model() was called.")

def get_named_symbol_in_list(l, name):
    for h in l:
        if h.name == name:
            return h
    return None

def get_node_in(element, name: str):
    if isinstance(element, GbomlGraph):
        l = element.list_nodes
    elif isinstance(element, Node):
        l = element.nodes
    else:
        raise Exception(f"Unknown class {element.__class__}")
    return get_named_symbol_in_list(l, name)
    

def get_hyperedge_in(element, name: str):
    if isinstance(element, GbomlGraph):
        l = element.list_hyperedges
    elif isinstance(element, Node):
        l = element.hyperedges
    else:
        raise Exception(f"Unknown class {element.__class__}")
    return get_named_symbol_in_list(l, name)

def update_parameter(symb, param):
    symb.parameters = [x if x.name != param.name else param for x in symb.parameters] 

### Set name period


In [3]:
def set_periodicity(TimeHorizon):
    
    if TimeHorizon >= 8760:
        if TimeHorizon % 8760 == 0:
            time = f"_{int(TimeHorizon/8760)}years.json"

        elif TimeHorizon % 8760 != 0:
            time = f"_{TimeHorizon}hours.json"
                
    if TimeHorizon < 8760:
            if TimeHorizon % 24 == 0:
                time = f"_{int(TimeHorizon/24)}days.json"
            
            else:
                time = f"_{TimeHorizon}hours.json"
                
    return time

### Change parameters

In [4]:
def update_node_parameter(model, nodes_to_modify, new_capacity_value):
    """
    Updates the parameter for a list of nodes in a GBOML model.

    :param model: The loaded GBOML model (MyGbomlGraph)
    :param nodes_to_modify: List of node names (strings) to modify
    :param new_capacity_value: New value to assign to the 'max_capacity' parameter
    """
    for node_name in nodes_to_modify:
        node = model.get_node_by_name(node_name)
        if node is not None:
            param_updated = Parameter(name="max_capacity", value=new_capacity_value)
            update_parameter(node, param_updated)
            print(f"✅ max_capacity of {node_name} set to {new_capacity_value}")
        else:
            print(f"⚠️ Node {node_name} not found in the model.")
            
            
def update_subnode_parameter(model, cluster_name, subnode_names, param_name, new_value):
    """
    Updates a parameter for a list of subnodes within a given cluster node in a GBOML model.

    :param model: The loaded GBOML model (MyGbomlGraph)
    :param cluster_name: Name of the cluster (e.g., 'INLAND')
    :param subnode_names: List of subnode names (e.g., ['DME_PLANTS'])
    :param param_name: Parameter to update (e.g., 'max_capacity')
    :param new_value: Value to assign (should be a number)
    """
    cluster = model.get_node_by_name(cluster_name)
    if cluster is None:
        print(f"❌ Cluster node '{cluster_name}' not found.")
        return

    for subnode in cluster.nodes:
        if subnode.name in subnode_names:
            param = Parameter(param_name, Expression("literal", new_value))  # ✅ Fix: no str()
            update_parameter(subnode, param)
            print(f"✅ {cluster_name}.{subnode.name}: {param_name} set to {new_value}")


### Build model

In [5]:
def build_model(TimeHorizon,model_name):
    program = parse_file(model_name)
    model = MyGbomlGraph()
    model.add_nodes_in_model(*deepcopy(program.get_nodes()))
    model.add_hyperedges_in_model(*deepcopy(program.get_links()))
    model.add_global_parameters(deepcopy(program.get_global_parameters()))
    model.set_timehorizon(TimeHorizon)

    return model


def build_and_solve(model, outfile=None):
    """
    Builds and solves the given GBOML model using Gurobi.
    If the solution is optimal, it exports a JSON and CSV with results.
    If infeasible, it prints the issue and exports a .lp file for debugging.

    :param model: GBOML model (MyGbomlGraph)
    :param outfile: Optional output filename (JSON). CSV will use same name.
    :return: Dictionary of results (or empty if infeasible)
    """
    print("🚧 Building model...")
    model.build_model(8)

    # Export LP version of the model for manual debug if needed
    model.export_model_to_lp("debug_model.lp")
    print("📤 Model exported to 'debug_model.lp'")

    print("🚀 Solving...")
    solution, objective, status, solver_info, cai, vai = model.solve_gurobi(
        details=os.path.join(model_folder, "gurobi_detail.txt"),
        opt_file=os.path.join(model_folder, "gurobi.opt")
    )

    print("🧠 Gurobi status:", status)
    if "OPTIMAL" not in str(status).upper():
        print("❌ No feasible solution found.")
        return {}

    print("✅ Optimal solution found.")

    out = model.turn_solution_to_dictionary(solver_info, status, solution, objective, cai, vai)
    outlist = model.turn_solution_to_list(solution, constraints_info=cai)
    list_names = [x[0] for x in outlist]
    list_val = [x[1] for x in outlist]

    if outfile:
        with open(outfile, 'w') as f_json:
            json.dump(out, f_json, indent=4, sort_keys=True)
        write_csv(outfile.replace(".json", ".csv"), list_names, list_val, transpose=True)
        print(f"📁 Results saved: {outfile} and CSV version.")

    return out

### Scenario maker


In [6]:
def scenario_maker(clusters, nodes, params):
    """
    Generates a list of dictionaries (modifs) to apply parameter modifications
    to a combination of clusters and subnodes in a GBOML model.

    :param clusters: list of cluster names (e.g., ["INLAND", "ZEEBRUGGE"])
    :param nodes: list of subnode names (e.g., ["DME_PLANTS", "ETH_PLANTS"])
    :param params: dictionary of parameters to modify (e.g., {"max_capacity": 0})
    :return: list of formatted dictionaries for scenario modification
    """
    modifs = []
    for cluster in clusters:
        for node in nodes:
            modifs.append({
                "cluster": cluster,
                "node": node,
                "params": params.copy()
            })
    return modifs



def run_scenarios(base_model, file_path, period, new_scenario=None, modifications=None, scenario_name=None):
    """
    Runs a GBOML simulation scenario. Optionally modifies certain node parameters.

    :param base_model: Compiled GBOML model (MyGbomlGraph)
    :param file_path: Path to save output files (with trailing slash)
    :param period: Suffix for the output file (e.g. _2030)
    :param new_scenario: Optional name of the scenario (used as suffix in filenames)
    :param modifications: Optional list of parameter changes (only if new_scenario is not None). Format:
                          [{"cluster": "INLAND", "node": "DME_PLANTS", "params": {"max_capacity": 0}}]
    """
    from copy import deepcopy
    import time

    if new_scenario:
        scenario = new_scenario
    elif scenario_name:
        scenario = scenario_name
    else:
        scenario = "base" 
    
    print(f"\n🔁 Running scenario: " + colored(f'{scenario}', 'white', 'on_blue'))
    start_time = time.time()

    model = deepcopy(base_model)

    if new_scenario and modifications:
        for item in modifications:
            cluster_name = item["cluster"]
            node_name = item["node"]
            for param_name, value in item["params"].items():
                try:
                    update_subnode_parameter(model, cluster_name, [node_name], param_name, value)
                except Exception as e:
                    print(colored(f"⚠️ Skipped modification: {cluster_name}.{node_name}.{param_name} – {e}", 'yellow'))

    out_path = os.path.join(file_path, f"scenario_{scenario}_{period}.json")
    results = build_and_solve(model, outfile=out_path)

    duration = time.time() - start_time
    print(f"✅ Scenario '{scenario}' completed in {duration:.1f} seconds.")

    return results

# MAIN

- Timehorizon setting

In [7]:
TimeHorizon = 8760
# TimeHorizon = 1000 # 1 week
# TimeHorizon = 24*7 # 1 week

period = set_periodicity(TimeHorizon)

### Model 3 clusters from article

In [8]:
file_3_clusters_efuel_INLAND = "be_model_3_clusters_all_efuels.txt" 

model_3_clusters_efuel_INLAND = build_model(TimeHorizon, os.path.join(model_folder, file_3_clusters_efuel_INLAND)) 

In [9]:
three_clusters_efuel_INLAND_scenario = run_scenarios(model_3_clusters_efuel_INLAND, file_path=simulation_folder, period=period, scenario_name='efuel_INLAND_ZEEBRUGGE')


🔁 Running scenario: efuel_INLAND_ZEEBRUGGE
🚧 Building model...
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node DEMAND : --- 0.0 seconds ---
Check variables of node BALANCE : --- 0.0 seconds ---
Check variables of node CO2_EXPORT : --- 0.0010025501251220703 seconds ---
Check variables of node CO2_STORAGE : --- 0.0 seconds ---
Check variables of node PCCC_SMR : --- 0.0 seconds ---
Check variables of node PCCC_OCGT : --- 0.0009965896606445312 seconds ---
Check variables of node PCCC_CCGT : --- 0.0 seconds ---
Check variables of node PCCC_CHP : --- 0.0006413459777832031 seconds ---
Check variables of node PCCC_WS : --- 0.0002090930938720703 seconds ---
Check variables of node PCCC_BM : --- 0.0 seconds ---
Check variables of node DAC : --- 0.0 seconds ---
Check variables of node PCCC_MEOH : --- 0.0 seconds ---
Check variables of node PCCC_FT : --- 0.0006833076477050781 seconds ---
Check variables of node PCCC_DME : --- 0.0 seconds ---
Check variables of node ME

In [10]:
clusters = ["INLAND", "ZEEBRUGGE"]
nodes = [ "FT_PLANTS", "REFINERY", "REFINED_PETROL_STORAGE", "PETROL_STORAGE"]
# nodes = ["ETH_PLANTS","BIOETH_PLANTS", "CELLETH_PLANTS", "ETHANOL_STORAGE"]
params = {"max_capacity": 0, "pre_installed_capacity": 0}

modifs = scenario_maker(clusters, nodes, params)
modifs_table = pf.transform_dict_into_table_several_column(modifs)

,cluster,node,params
0,INLAND,FT_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
1,INLAND,REFINERY,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
2,INLAND,REFINED_PETROL_STORAGE,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
3,INLAND,PETROL_STORAGE,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
4,ZEEBRUGGE,FT_PLANTS,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
5,ZEEBRUGGE,REFINERY,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
6,ZEEBRUGGE,REFINED_PETROL_STORAGE,"{'max_capacity': 0, 'pre_installed_capacity': 0}"
7,ZEEBRUGGE,PETROL_STORAGE,"{'max_capacity': 0, 'pre_installed_capacity': 0}"


In [11]:
three_clusters_efuel_INLAND_scenario = run_scenarios(model_3_clusters_efuel_INLAND, file_path=simulation_folder, period=period, new_scenario="efuel_INLAND_ZEEBRUGGE_no_ft", modifications=modifs)


🔁 Running scenario: efuel_INLAND_ZEEBRUGGE_no_ft
✅ INLAND.FT_PLANTS: max_capacity set to 0
✅ INLAND.FT_PLANTS: pre_installed_capacity set to 0
✅ INLAND.REFINERY: max_capacity set to 0
✅ INLAND.REFINERY: pre_installed_capacity set to 0
✅ INLAND.REFINED_PETROL_STORAGE: max_capacity set to 0
✅ INLAND.REFINED_PETROL_STORAGE: pre_installed_capacity set to 0
✅ INLAND.PETROL_STORAGE: max_capacity set to 0
✅ INLAND.PETROL_STORAGE: pre_installed_capacity set to 0
✅ ZEEBRUGGE.FT_PLANTS: max_capacity set to 0
✅ ZEEBRUGGE.FT_PLANTS: pre_installed_capacity set to 0
✅ ZEEBRUGGE.REFINERY: max_capacity set to 0
✅ ZEEBRUGGE.REFINERY: pre_installed_capacity set to 0
✅ ZEEBRUGGE.REFINED_PETROL_STORAGE: max_capacity set to 0
✅ ZEEBRUGGE.REFINED_PETROL_STORAGE: pre_installed_capacity set to 0
✅ ZEEBRUGGE.PETROL_STORAGE: max_capacity set to 0
✅ ZEEBRUGGE.PETROL_STORAGE: pre_installed_capacity set to 0
🚧 Building model...
Check variables of node INLAND : --- 0.0 seconds ---
Check variables of node DEMAND : 